In [145]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Heatmap, TileLayer
from ipywidgets import AppLayout
from ipywidgets import HTML, Layout, Dropdown, Output, Textarea, VBox, Label
import bqplot as bq
import numpy as np
from pandas import date_range
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed
  
from random import choice

In [146]:
#!pip install earthengine-api
# Installs geemap package
#import subprocess

#try:
import geemap
#except ImportError:
#    print('Installing geemap ...')
#    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
import ee
import geemap

In [147]:
Map = geemap.Map(center=(0.036, 102.0157), zoom=13)


In [148]:
from ipywidgets import interact, widgets
from IPython.display import display

start = widgets.Text(
    value='2020-01-01',
    placeholder='Choose Start Date!',
    description='Start Date:',
    disabled=False
)





In [149]:
from ipywidgets import interact, widgets
from IPython.display import display

end = widgets.Text(
    value='2020-12-31',
    placeholder='Choose End Date!',
    description='End Date:',
    disabled=False
)





In [150]:
tab = widgets.IntSlider(
    value=3,
    min=0,
    max=100,
    step=1,
    description='Cloud Cover Filter:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)



In [151]:
# Create and output new int variable with value of slider
new_variable = int(tab.value)


In [152]:

#Definisikan Daerah Penelitian

geometry2 = ee.Geometry.Polygon(
        [[[101.93336117527792, 0.06990952997337596],
          [101.93336117527792, -0.06707621616689373],
          [102.0696603085787, -0.06707621616689373],
          [102.0696603085787, 0.06990952997337596]]])
sctrA = ee.FeatureCollection("users/rifkynauvalhsp/healthmonitoring/SectorA")
def maskS2clouds(img):
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  # Get the pixel QA band.
  qa = img.select('QA60')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return img.updateMask(mask).divide(10000)

def addindices(image): 
  ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi']);
  gndvi = image.normalizedDifference(['B2', 'B4']).rename(['gndvi']);
  ndmi = image.normalizedDifference(['B8', 'B11']).rename(['ndmi']);
  savi = image.expression('(NIR-RED)/(NIR+RED+0.5)*(1.5)',{
  'NIR' : image.select('B8'),
  'RED':image.select('B4')
}).rename('savi')
  evi = image.expression('2.5*((NIR-RED)/(NIR+(RED*2.4)+1))',{
  'NIR' : image.select('B8'),
  'RED':image.select('B4')
}).rename('evi')
  
  return image.addBands(ndvi).addBands(gndvi).addBands(ndmi).addBands(savi).addBands(evi)


dataset = ee.ImageCollection('COPERNICUS/S2_SR').select(['QA60','B2','B3','B4','B8','B11']).filterBounds(sctrA).filterDate(start.value, end.value).filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', new_variable).map(maskS2clouds).map(addindices);
image = dataset.mean()
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}, "image")


    

In [153]:
mean = ee.Number(image.select('ndvi').reduceRegion(**{'geometry': geometry2,
'reducer': ee.Reducer.mean(),'scale': 30,
'maxPixels': 1e9
}).values().get(0))

standev = ee.Number(image.select('ndvi').reduceRegion(**{'geometry': geometry2,
'reducer': ee.Reducer.stdDev(),'scale': 30,
'maxPixels': 1e9
}).values().get(0))

zscore = image.expression('((ndvi-mean)/standev)',{
  'ndvi' : image.select('ndvi'),
  'mean':mean,
  'standev':standev,
}).rename('zscore')
    
image = image.addBands(zscore)

In [154]:
# Load watersheds from a Fusion Table.
sctrA = ee.FeatureCollection("users/rifkynauvalhsp/healthmonitoring/SectorA")
sctrB = ee.FeatureCollection("users/rifkynauvalhsp/healthmonitoring/SectorB")
sctrC = ee.FeatureCollection("users/rifkynauvalhsp/healthmonitoring/SectorC")
sctrD = ee.FeatureCollection("users/rifkynauvalhsp/healthmonitoring/SectorD")
# Add the mean of each image as new properties of each feature.


# This function computes the feature's geometry area and adds it as a property.
def addAreaA(feature):
  
  result = image.reduceRegion(**{
    'reducer':ee.Reducer.mean(),
  'geometry': feature.geometry(),
  'scale': 30,
  'crs': 'EPSG:5070',
  'bestEffort': True,
  'maxPixels': 1e12,
 
  })
 
  feature = feature.set({'Estate': 'A'})
  feature = feature.set({'RED': result.get('B4')})
  feature = feature.set({'GREEN': result.get('B3')})
  feature = feature.set({'BLUE': result.get('B2')})
  feature = feature.set({'NIR': result.get('B8')}) 
  feature = feature.set({'NDVI': result.get('ndvi')})
  feature = feature.set({'GNDVI': result.get('gndvi')})
  feature = feature.set({'NDMI': result.get('ndmi')})
  feature = feature.set({'SAVI': result.get('savi')})
  feature = feature.set({'EVI': result.get('evi')})
  feature = feature.set({'zscore': result.get('zscore')})
  #feature = feature.set({'Significance':  ee.Algorithms.If(ee.Number.parse((result.get('Zscore').values()) > ee.Number.parse(-1.96), "Significance","Not Significance"))});
  return feature
# This function computes the feature's geometry area and adds it as a property.
def csvInfoA1(feature):
  feature = feature.set({'class': 'Healthy'})
  return feature

def csvInfoA2(feature):
  feature = feature.set({'class': 'Unhealthy'})
  return feature

# Map the area getting function over the FeatureCollection.
areaAddedA = sctrA.map(addAreaA)



# Print the first feature from the collection with the added property.
#print('First feature: \n')
#pprint(areaAdded.limit(50).getInfo())
HealthyA = areaAddedA.filter(ee.Filter.gt('NDVI', 0.7))
UnhealthyA = areaAddedA.filter(ee.Filter.lt('NDVI', 0.7))
HealthyA1 = HealthyA.map(csvInfoA1)
UnhealthyA1 = UnhealthyA.map(csvInfoA2)
#numHealthyA = HealthyA.size().getInfo()
#numUnhealthyA = UnhealthyA.size().getInfo()
#conditionalA =  ee.Algorithms.If(numHealthyA > numUnhealthyA, 'green',  'red' )
#colorsA = conditionalA.getInfo()
#Map.addLayer(sctrA,{'color': colorsA}, 'Estate A')

In [155]:
# This function computes the feature's geometry area and adds it as a property.
def addAreaB(feature):
  
  result = image.reduceRegion(**{
    'reducer':ee.Reducer.mean(),
  'geometry': feature.geometry(),
  'scale': 30,
  'crs': 'EPSG:5070',
  'bestEffort': True,
  'maxPixels': 1e12,
 
  })
 
  feature = feature.set({'Estate': 'B'})
  feature = feature.set({'RED': result.get('B4')})
  feature = feature.set({'GREEN': result.get('B3')})
  feature = feature.set({'BLUE': result.get('B2')})
  feature = feature.set({'NIR': result.get('B8')}) 
  feature = feature.set({'NDVI': result.get('ndvi')})
  feature = feature.set({'GNDVI': result.get('gndvi')})
  feature = feature.set({'NDMI': result.get('ndmi')})
  feature = feature.set({'SAVI': result.get('savi')})
  feature = feature.set({'EVI': result.get('evi')})
  feature = feature.set({'zscore': result.get('zscore')})
  return feature

areaAddedB = sctrB.map(addAreaB)

HealthyB = areaAddedB.filter(ee.Filter.gt('NDVI', 0.7))
UnhealthyB = areaAddedB.filter(ee.Filter.lt('NDVI', 0.7))
HealthyB1 = HealthyB.map(csvInfoA1)
UnhealthyB1 = UnhealthyB.map(csvInfoA2)
#numHealthyB = HealthyB.size().getInfo()
#numUnhealthyB = UnhealthyB.size().getInfo()
#conditionalB =  ee.Algorithms.If(numHealthyB > numUnhealthyB, 'green',  'red' )
#colorsB = conditionalB.getInfo()
#Map.addLayer(HealthyB, {'color': 'green'}, 'Healthy B');
#Map.addLayer(UnhealthyB, {'color': 'red'}, 'Unhealthy B');
#Map.addLayer(sctrB,{'color': colorsA}, 'Estate B')


In [156]:
# This function computes the feature's geometry area and adds it as a property.
def addAreaC(feature):
  
  result = image.reduceRegion(**{
    'reducer':ee.Reducer.mean(),
  'geometry': feature.geometry(),
  'scale': 30,
  'crs': 'EPSG:5070',
  'bestEffort': True,
  'maxPixels': 1e12,
 
  })
 
  feature = feature.set({'Estate': 'C'})
  feature = feature.set({'RED': result.get('B4')})
  feature = feature.set({'GREEN': result.get('B3')})
  feature = feature.set({'BLUE': result.get('B2')})
  feature = feature.set({'NIR': result.get('B8')}) 
  feature = feature.set({'NDVI': result.get('ndvi')})
  feature = feature.set({'GNDVI': result.get('gndvi')})
  feature = feature.set({'NDMI': result.get('ndmi')})
  feature = feature.set({'SAVI': result.get('savi')})
  feature = feature.set({'EVI': result.get('evi')})
  feature = feature.set({'zscore': result.get('zscore')})
  return feature

areaAddedC = sctrC.map(addAreaC)
HealthyC = areaAddedC.filter(ee.Filter.gt('NDVI', 0.7))
UnhealthyC = areaAddedC.filter(ee.Filter.lt('NDVI', 0.7))
HealthyC1 = HealthyC.map(csvInfoA1)
UnhealthyC1 = UnhealthyC.map(csvInfoA2)
#numHealthyC = HealthyC.size().getInfo()
#numUnhealthyC = UnhealthyC.size().getInfo()
#conditionalC =  ee.Algorithms.If(numHealthyC > numUnhealthyC, 'green',  'red' )
#colorsC = conditionalC.getInfo()
#Map.addLayer(HealthyC, {'color': 'green'}, 'Healthy C');
#Map.addLayer(UnhealthyC, {'color': 'red'}, 'Unhealthy C');
#Map.addLayer(sctrC,{'color': colorsA}, 'Estate C')


In [157]:
# This function computes the feature's geometry area and adds it as a property.
def addAreaD(feature):
  
  result = image.reduceRegion(**{
    'reducer':ee.Reducer.mean(),
  'geometry': feature.geometry(),
  'scale': 30,
  'crs': 'EPSG:5070',
  'bestEffort': True,
  'maxPixels': 1e12,
 
  })
 
  feature = feature.set({'Estate': 'D'})
  feature = feature.set({'RED': result.get('B4')})
  feature = feature.set({'GREEN': result.get('B3')})
  feature = feature.set({'BLUE': result.get('B2')})
  feature = feature.set({'NIR': result.get('B8')}) 
  feature = feature.set({'NDVI': result.get('ndvi')})
  feature = feature.set({'GNDVI': result.get('gndvi')})
  feature = feature.set({'NDMI': result.get('ndmi')})
  feature = feature.set({'SAVI': result.get('savi')})
  feature = feature.set({'EVI': result.get('evi')})
  feature = feature.set({'zscore': result.get('zscore')})
  return feature


areaAddedD = sctrD.map(addAreaD)
HealthyD = areaAddedD.filter(ee.Filter.gt('NDVI', 0.7))
UnhealthyD = areaAddedD.filter(ee.Filter.lt('NDVI', 0.7))
HealthyD1 = HealthyD.map(csvInfoA1)
UnhealthyD1 = UnhealthyD.map(csvInfoA2)
kesehatantanaman = HealthyA1.merge(UnhealthyA1).merge(HealthyB1).merge(UnhealthyB1).merge(HealthyC1).merge(UnhealthyC1).merge(HealthyD1).merge(UnhealthyD1)
# Check the number of watersheds after filtering for size and location.
#numHealthyD = HealthyD.size().getInfo()
#numUnhealthyD = UnhealthyD.size().getInfo()
#conditionalD =  ee.Algorithms.If(numHealthyD > numUnhealthyD, 'green',  'red' )
#colorsD = conditionalD.getInfo()
#Map.addLayer(HealthyD, {'color': 'green'}, 'Healthy D');
#Map.addLayer(UnhealthyD, {'color': 'red'}, 'Unhealthy D');
#Map.addLayer(sctrD,{'color': colorsD}, 'Estate D')

In [158]:
rd = widgets.RadioButtons(
    options=[('A', HealthyA), ('B', HealthyB), ('C', HealthyC),('D', HealthyD)],
#     value='pineapple',
    description='Estate:',
    disabled=False
)
rd1 = widgets.RadioButtons(
    options=[('A', UnhealthyA), ('B', UnhealthyB), ('C', UnhealthyC),('D', UnhealthyD)],
#   value='pineapple',
    description='Estate:',
    disabled=False
)

In [159]:
btn1 = widgets.Button(
    value=False,
    description='Process Analysis',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
   
Map.addLayer(rd.value, {'color': 'green'}, 'Healthy');
Map.addLayer(rd1.value, {'color': 'red'}, 'Unhealthy');
btn1.on_click(Map.addLayer(rd.value, {'color': 'green'}, 'Healthy'),Map.addLayer(rd1.value, {'color': 'red'}, 'Unhealthy')) 


 



In [160]:
btn2 = widgets.Button(
    value=False,
    description='Download Analysis',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
task = ee.batch.Export.table.toDrive(**{
      'collection': kesehatantanaman,
      'description':'EVI6',
      'fileFormat': 'CSV',
      'selectors' : ['Estate','Compartmen', 'RED','GREEN','BLUE','NIR','NDVI', 'GNDVI', 'EVI','NDMI', 'SAVI', 'GNDVI', 'EVI','class','zscore'] 
    })

def data2(c):   
    task.start()
btn2.on_click(data2)  


In [161]:
#from __future__ import print_function
#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

#def f(x):
#    return x
#interact(f, x=10);



In [162]:
header = HTML("<h1>Plant Health Monitoring App</h1>", layout=Layout(height='auto'))
header.style.text_align='center'




In [163]:
out = HTML(
    value='',
    layout=Layout(width='auto', height='auto')
)

In [164]:
from ipywidgets import GridspecLayout

grid = GridspecLayout(5, 4, height='1000px')
grid[:4, 1:] = Map
grid[:, 0] = VBox([header,Label("Input Start Date:"),start,Label("Input End Date:"),end,Label("Input Cloud Filter:"),tab,Label("Display Healthy Plant:"),
                             rd,Label("Display Unealthy Plant:"),rd1,btn1,btn2])

In [165]:
grid

GridspecLayout(children=(Map(center=[0.036, 102.0157], controls=(WidgetControl(options=['position'], widget=HB…